# Build the Neural Network

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Getting the device

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'Using {device} device')

Using cuda device


## Defining the Class

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Instance of NeuralNetwork, and move it to the device, and print its structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call model.forward() directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.

In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)        # argmax: Returns the indices of the maximum values along an axis.
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


## Model Layers

In [6]:
input_image = torch.rand(3,28,28)     # minibatch of 3 images
print(input_image.size())

torch.Size([3, 28, 28])


nn.FLattern:

We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear:

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU:

Non-linear activations are what create the complex mappings between the model’s inputs and outputs.

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1852,  0.1554, -0.1608, -0.0283, -0.2780, -0.4456,  0.3936, -0.1174,
          0.1329,  0.2249, -0.1454,  0.1055,  0.2095,  0.6123, -0.1240,  0.0805,
          0.3994, -0.4189,  0.0070,  0.1439],
        [-0.3239, -0.2685,  0.0305,  0.0741, -0.7453, -0.4577,  0.4445,  0.0752,
          0.5215,  0.1108, -0.1998,  0.6682, -0.0658,  0.5449, -0.2059, -0.2388,
          0.3574, -0.7588, -0.1435,  0.0522],
        [-0.2038, -0.2589, -0.2250, -0.1682, -0.4074, -0.1774,  0.2514, -0.2915,
          0.4959,  0.0973, -0.2580,  0.3216, -0.1458,  0.2976, -0.3054, -0.0915,
          0.4205, -0.2944,  0.1246, -0.0702]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1852, 0.1554, 0.0000, 0.0000, 0.0000, 0.0000, 0.3936, 0.0000, 0.1329,
         0.2249, 0.0000, 0.1055, 0.2095, 0.6123, 0.0000, 0.0805, 0.3994, 0.0000,
         0.0070, 0.1439],
        [0.0000, 0.0000, 0.0305, 0.0741, 0.0000, 0.0000, 0.4445, 0.0752, 0.5215,
         0.1108, 0.0000, 0.6682, 0.0000, 0.5449, 0.00

nn.Sequential:

nn.Sequential is an ordered container of modules.

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)   # The last linear layer of the neural network returns logits - raw values in [-infty, infty]

nn.Softmax:

The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0338, -0.0254, -0.0116,  ...,  0.0168, -0.0308, -0.0019],
        [ 0.0257, -0.0342,  0.0262,  ...,  0.0217, -0.0240, -0.0115]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0037,  0.0233], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0374,  0.0168, -0.0431,  ..., -0.0327,  0.0403,  0.0136],
        [-0.0407, -0.0066, -0.0313,  ...,  0.0092, -0.0006,  0.0433]],
       device='cuda:0', grad_fn=<Sl

## Docs

https://pytorch.org/docs/stable/nn.html